In [1]:
import tensorflow as tf

In [2]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=10000)

In [3]:
max_len = 512
num_classes = 2

X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_len)
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=max_len)

y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [4]:
class Inception(tf.keras.Model):

    def __init__(self):
        super().__init__()

        self.p1_1 = tf.keras.layers.Conv1D(32, 1, activation='relu')

        self.p2_1 = tf.keras.layers.Conv1D(128, 1, activation='relu', dilation_rate=2)
        self.p2_2 = tf.keras.layers.Conv1D(64, 3, padding='same', activation='relu', dilation_rate=3)

        self.p3_1 = tf.keras.layers.Conv1D(64, 1, activation='relu')
        self.p3_2 = tf.keras.layers.Conv1D(32, 5, padding='same', activation='relu')

        self.p4_1 = tf.keras.layers.MaxPool1D(3, 1, padding='same')
        self.p4_2 = tf.keras.layers.Conv1D(64, 1, activation='relu')

    def call(self, x):
        p1 = self.p1_1(x)
        p2 = self.p2_2(self.p2_1(x))
        p3 = self.p3_2(self.p3_1(x))
        p4 = self.p4_2(self.p4_1(x))

        return tf.keras.layers.Concatenate()([p1, p2, p3, p4])

In [5]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Embedding(input_dim=10000, output_dim=512))

model.add(tf.keras.layers.Conv1D(256, 1, strides=2, padding='same', activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.MaxPool1D(pool_size=3, strides=2, padding='same'))

model.add(Inception())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.MaxPool1D(pool_size=3, strides=2, padding='same'))

model.add(tf.keras.layers.GlobalAvgPool1D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(2, activation="relu"))

In [6]:
input_shape=(512, 1)
model.build(input_shape)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (512, 1, 512)               │       5,120,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ (512, 1, 256)               │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (512, 1, 256)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (512, 1, 256)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ inception (Inception)                │ (512, 1, 192)               │         108,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (512, 1, 192)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_2 (MaxPooling1D)       │ (512, 1, 192)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (512, 192)                  │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (512, 192)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (512, 2)                    │             386 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,360,642 (20.45 MB)

 Trainable params: 5,360,642 (20.45 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
lr = 0.1
num_epochs = 10
batch_size = 128

In [8]:
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"])

model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=num_epochs,
    validation_split=0.1)

Epoch 1/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 27s 87ms/step - accuracy: 0.6238 - loss: 0.7935 - val_accuracy: 0.7424 - val_loss: 1.0579
Epoch 2/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.8370 - loss: 0.4623 - val_accuracy: 0.8300 - val_loss: 0.9644
Epoch 3/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.8936 - loss: 0.3562 - val_accuracy: 0.8428 - val_loss: 0.7645
Epoch 4/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.9154 - loss: 0.3129 - val_accuracy: 0.7548 - val_loss: 1.9649
Epoch 5/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 10s 30ms/step - accuracy: 0.8718 - loss: 0.8933 - val_accuracy: 0.8256 - val_loss: 0.7933
Epoch 6/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 10s 30ms/step - accuracy: 0.9187 - loss: 0.3073 - val_accuracy: 0.8316 - val_loss: 0.9102
Epoch 7/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - accuracy: 0.9263 - loss: 0.2678 - val_accuracy: 0.8252 - val_loss: 0.9637
Epoch 8/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 10s 30ms/step - accuracy: 0.9363 - loss: 0.2447 - val

In [9]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 1.4017027616500854
Test accuracy: 0.7894399762153625
